#### Keras Tuner- Decide Number of Hidden Layers And Neuron In Neural Network

In [14]:
# Keras Tuner Hyperparameter Tuning-How To Select Hidden Layers And Number of Hidden Neurons In ANN

In [15]:
# https://www.youtube.com/watch?v=Lx16T9cl5ng&list=PLZoTAELRMXVPGU70ZGsckrMdr0FteeRUi&index=19
# https://github.com/krishnaik06/Keras-Tuner

In [16]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [17]:
# df=pd.read_csv('Real_Combine.csv', dtype=np.float64)
df=pd.read_csv('Real_Combine.csv')

In [18]:
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [19]:
X=df.iloc[:,:-1] ## independent features
y=df.iloc[:,-1] ## dependent features

#### Hyperparameters
1. How many number of hidden layers we should have?
2. How many number of neurons we should have in hidden layers?
3. Learning Rate 

In [20]:
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(layers.Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='mean_absolute_error',
        metrics=['mean_absolute_error'])
    return model

In [21]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')

INFO:tensorflow:Reloading Oracle from existing project project\Air Quality Index\oracle.json


In [22]:
tuner.search_space_summary()

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [24]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

 1/24 [>.............................] - ETA: 0s - loss: 132.2704 - mean_absolute_error: 132.2704

NotFoundError: Failed to create a directory: project\Air Quality Index\trial_d5e647fd86d4075ac01dd1bf9c7be5b9\checkpoints\epoch_0\checkpoint_temp_322d812ad2f541199df248e25f9578ad; No such file or directory [Op:SaveV2]

In [25]:
tuner.results_summary()